### this script cleans and compiles raw data. 

Input includes:
- **pub**: discipline and year information of publications
- **pub_fund**:funding instance of publications
- **pub_author**:author country of publications
- **cntry**: country convert table which map wos country name to standard format.

Output includes:
- **clean_pub**: a consolidated pickle file has abovementioned information with two extra indicators including funded (whether the paper is funded) and intcol (whether the paper is internationally coauthored).

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [2]:
pub_path="../../data/nf_folder/Data/RawData/pubs.csv"
pub_fund_path="../../data/nf_folder/Data/RawData/pub_funder.csv"
pub_author_path="../../data/nf_folder/Data/RawData/pub_author.csv"
cntry_path="../../data/NationalFunding/Data/AdditionalData/cntry_convert.xlsx"

clean_pub_path="../../data/NationalFunding/Data/DerivedData/CleanedRawData/pub.pkl"

In [3]:
# Parameters
pub_path = "../data/NationalFunding/Data/simulated_data/Data/RawData/pubs.csv"
pub_author_path = (
    "../data/NationalFunding/Data/simulated_data/Data/RawData/pub_author.csv"
)
pub_fund_path = (
    "../data/NationalFunding/Data/simulated_data/Data/RawData/pub_funder.csv"
)
cntry_path = (
    "../data/NationalFunding/Data/simulated_data/Data/AdditionalData/cntry_convert.xlsx"
)
clean_pub_path = "../data/NationalFunding/Data/simulated_data/Data/DerivedData/CleanedRawData/pub.pkl"


In [4]:
pd.DataFrame().to_pickle(clean_pub_path)

In [5]:
pub_df=pd.read_csv(pub_path,sep="\t")
pub_df.head()

,ID_Art,Annee_Bibliographique,Code_Discipline,Code_Document
0,0,2014,29,1
1,1,2017,32,1
2,2,2012,15,1
3,3,2010,53,1
4,4,2015,22,1


count how many pubs in each document type

In [6]:
pub_fund=pd.read_csv(pub_fund_path,sep="\t", on_bad_lines='skip')
pub_fund.head()

,ID_Art,Country
0,253668,CZECH-REPUBLIC
1,574338,PEOPLES-R-CHINA
2,136486,BRAZIL
3,513654,USA
4,348664,USA


count how many fa-gn entries are with grant number

In [7]:
pub_author=pd.read_csv(pub_author_path,sep="\t", on_bad_lines='skip')
pub_author.head()

,ID_Art,Pays
0,614376,SOUTH-KOREA
1,330068,SWITZERLAND
2,324252,USA
3,299648,USA
4,571067,USA


In [8]:
cntry_df=pd.read_excel(cntry_path)
cntry_dict=dict(zip(cntry_df['Clarivate country'],cntry_df['Country']))

change the clarivate name into the standard format

In [9]:
pub_fund=pub_fund.replace(to_replace={'Country':cntry_dict})

In [10]:
pub_author=pub_author.replace(to_replace={'Pays':cntry_dict})

consolidate three datasets into one big data sheet

In [11]:
pub_fund=pub_fund[['ID_Art','Country']]
pub_fund=pub_fund.groupby(['ID_Art'])['Country'].apply(list).reset_index()
pub_author=pub_author.groupby(['ID_Art'])['Pays'].apply(list).reset_index()

In [12]:
pub_author.head()

,ID_Art,Pays
0,0,[Canada]
1,1,"[Russia, Spain]"
2,2,[Turkey]
3,3,"[United States, Iran]"
4,4,[Denmark]


In [13]:
pub_fund.head()

,ID_Art,Country
0,0,"[United States, Brazil, United States, Republi..."
1,1,"[Austria, China, China, China, United States, ..."
2,3,[China]
3,5,"[Austria, China, Taiwan, South Korea, Switzerl..."
4,8,"[Germany, United States, China, Germany, Unite..."


In [14]:
pub_df=pub_df.merge(pub_author,on='ID_Art',how='left')
pub_df=pub_df.merge(pub_fund,on='ID_Art',how='left')

In [15]:
pub_df['Country']=pub_df['Country'].fillna(value='Not-Funded')

In [16]:
pub_df.head()

,ID_Art,Annee_Bibliographique,Code_Discipline,Code_Document,Pays,Country
0,0,2014,29,1,[Canada],"[United States, Brazil, United States, Republi..."
1,1,2017,32,1,"[Russia, Spain]","[Austria, China, China, China, United States, ..."
2,2,2012,15,1,[Turkey],Not-Funded
3,3,2010,53,1,"[United States, Iran]",[China]
4,4,2015,22,1,[Denmark],Not-Funded


In [17]:
pub_df['author_distinct']=pub_df['Pays'].apply(lambda x: list(set(x)))
pub_df['funder_distinct']=pub_df['Country'].apply(lambda x:list(set(x)) 
                                                  if x!='Not-Funded' else 'Not-Funded')

In [18]:
pub_df=pub_df.rename(mapper={'ID_Art':'id','Annee_Bibliographique':'year','Code_Discipline':'dis',
                             'Code_Document':'type','Pays':'author','Country':'funder'},axis=1)

In [19]:
pub_df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct
0,0,2014,29,1,[Canada],"[United States, Brazil, United States, Republi...",[Canada],"[Republic of Serbia, China, Brazil, United Sta..."
1,1,2017,32,1,"[Russia, Spain]","[Austria, China, China, China, United States, ...","[Russia, Spain]","[China, United States, Austria, Russia, Japan]"
2,2,2012,15,1,[Turkey],Not-Funded,[Turkey],Not-Funded
3,3,2010,53,1,"[United States, Iran]",[China],"[Iran, United States]",[China]
4,4,2015,22,1,[Denmark],Not-Funded,[Denmark],Not-Funded


In [20]:
pub_df['funded']=pub_df['funder'].apply(lambda x:1 if x!='Not-Funded' else 0)
pub_df['IntCol']=pub_df['author_distinct'].apply(lambda x:1 if len(x)>1 else 0)
pub_df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol
0,0,2014,29,1,[Canada],"[United States, Brazil, United States, Republi...",[Canada],"[Republic of Serbia, China, Brazil, United Sta...",1,0
1,1,2017,32,1,"[Russia, Spain]","[Austria, China, China, China, United States, ...","[Russia, Spain]","[China, United States, Austria, Russia, Japan]",1,1
2,2,2012,15,1,[Turkey],Not-Funded,[Turkey],Not-Funded,0,0
3,3,2010,53,1,"[United States, Iran]",[China],"[Iran, United States]",[China],1,1
4,4,2015,22,1,[Denmark],Not-Funded,[Denmark],Not-Funded,0,0


In [21]:
def cofund(funders):
    label=1
    if funders=='Not-Funded':
        label=0
    elif 'EU' not in funders and len(funders)==1:
        label=0
    return label
pub_df['cofund']=pub_df['funder_distinct'].apply(lambda x: cofund(x))

In [22]:
def intfund(funder, author, cofund):
    label = 0
    if cofund==1:
        label =1
    elif funder!='Not-Funded' and funder[0] not in author:
        label = 1
    return label
pub_df['intfund']=pub_df[['funder_distinct','author_distinct','cofund']].apply(
    lambda x:intfund(x.funder_distinct,x.author_distinct,x.cofund),axis=1)

In [23]:
pub_df.to_pickle(clean_pub_path)